# Create a Custom Transformer

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /home/mickie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mickie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mickie/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [6]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

### Implement the StartingVerbExtractor class

In [22]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        print(sentence_list)
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))
            print(pos_tags)
            print('\n')

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

### Run program to test

In [23]:
def load_data():
    df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
    df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
    X = df.text.values
    y = df.category.values
    return X, y


def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline


def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = model_pipeline()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results(y_test, y_pred)

main()

['RT @wealthinsights: WSJ: Philanthropy as Enterprise http://ow.ly/Juyz']
[('rt', 'NN'), ('@', 'JJ'), ('wealthinsights', 'NNS'), (':', ':'), ('wsj', 'NN'), (':', ':'), ('philanthropy', 'NN'), ('a', 'DT'), ('enterprise', 'NN'), ('urlplaceholder', 'NN')]


['In Mexico, Danone Semilla project plans to create 4,000 jobs in microdistribution by 2015 #social_innovation http://t.co/7oWctKhH']
[('in', 'IN'), ('mexico', 'NN'), (',', ','), ('danone', 'NN'), ('semilla', 'NN'), ('project', 'NN'), ('plan', 'NN'), ('to', 'TO'), ('create', 'VB'), ('4,000', 'CD'), ('job', 'NN'), ('in', 'IN'), ('microdistribution', 'NN'), ('by', 'IN'), ('2015', 'CD'), ('#', '#'), ('social_innovation', 'NN'), ('urlplaceholder', 'NN')]


['Corporate Responsibility at Merck: $23.8 million investment in partnerships to address underlying barriers to health.', '#BeWell']
[('corporate', 'JJ'), ('responsibility', 'NN'), ('at', 'IN'), ('merck', 'NN'), (':', ':'), ('$', '$'), ('23.8', 'CD'), ('million', 'CD'), ('investment', 'N

[('the', 'DT'), ('new', 'JJ'), ('science', 'NN'), ('fair', 'JJ'), ('section', 'NN'), ('of', 'IN'), ('think', 'NN'), ('science', 'NN'), ('now', 'RB'), ('highlight', 'VBZ'), ('a', 'DT'), ('lot', 'NN'), ('of', 'IN'), ('fun', 'NN'), ('fact', 'NN'), ('about', 'IN'), ('science', 'NN'), ('and', 'CC'), ('medicine', 'NN'), ('urlplaceholder', 'NN')]


['RT Welcome to @PfizerLifeUK.', 'Pfizer Life provides tailored information, helping you to proactively manage your #health']
[('rt', 'NN'), ('welcome', 'NN'), ('to', 'TO'), ('@', 'VB'), ('pfizerlifeuk', 'NN'), ('.', '.')]


['William Ringo, Pfizer\x89Ûªs Senior Vice President, Worldwide Business Development, Strategy & Innovation, To Retire']
[('william', 'NN'), ('ringo', 'NN'), (',', ','), ('pfizer\x89ûªs', 'JJ'), ('senior', 'JJ'), ('vice', 'NN'), ('president', 'NN'), (',', ','), ('worldwide', 'NN'), ('business', 'NN'), ('development', 'NN'), (',', ','), ('strategy', 'NN'), ('&', 'CC'), ('innovation', 'NN'), (',', ','), ('to', 'TO'), ('retire', '

[('latino', 'NN'), ('usa', 'JJ'), ('anchor', 'NN'), ('maria', 'NNS'), ('hinojosa', 'VBP'), ('join', 'NN'), ('pfizer', 'NN'), ('in', 'IN'), ('hosting', 'VBG'), ('a', 'DT'), ('town', 'NN'), ('hall', 'NN'), ('in', 'IN'), ('la', 'NN'), ('to', 'TO'), ('focus', 'VB'), ('attention', 'NN'), ('on', 'IN'), ('cancer', 'NN'), ('in', 'IN'), ('the', 'DT'), ('latino', 'NN'), ('community', 'NN')]


['Recent studies indicate that 45% of residents of elderly homes are malnourished: http://t.co/AouKkLdt']
[('recent', 'JJ'), ('study', 'NN'), ('indicate', 'VBP'), ('that', 'IN'), ('45', 'CD'), ('%', 'NN'), ('of', 'IN'), ('resident', 'NN'), ('of', 'IN'), ('elderly', 'JJ'), ('home', 'NN'), ('are', 'VBP'), ('malnourished', 'VBN'), (':', ':'), ('urlplaceholder', 'NN')]


['This Global Community Day, 3,800 #CitiVolunteers in #Taiwan will give back by spending time w/ orphans &amp  underprivileged children.']
[('this', 'DT'), ('global', 'JJ'), ('community', 'NN'), ('day', 'NN'), (',', ','), ('3,800', 'CD'), ('#',

[('pfizer', 'NN'), ('global', 'JJ'), ('health', 'NN'), ('fellows', 'NNS'), ('&', 'CC'), ('amp', 'JJ'), ('accordia', 'NN'), ('global', 'JJ'), ('health', 'NN'), ('foundation', 'NN'), ('recognize', 'VB'), ('10', 'CD'), ('year', 'NN'), ('of', 'IN'), ('improving', 'VBG'), ('healthcare', 'NN'), ('in', 'IN'), ('africa', 'NN'), ('urlplaceholder', 'NN')]


['#Nestle CEO  A strong portfolio is driven by innovation, by putting the right resources behind the right things  2013 full-year results']
[('#', '#'), ('nestle', 'JJ'), ('ceo', 'NN'), ('a', 'DT'), ('strong', 'JJ'), ('portfolio', 'NN'), ('is', 'VBZ'), ('driven', 'VBN'), ('by', 'IN'), ('innovation', 'NN'), (',', ','), ('by', 'IN'), ('putting', 'VBG'), ('the', 'DT'), ('right', 'JJ'), ('resource', 'NN'), ('behind', 'IN'), ('the', 'DT'), ('right', 'JJ'), ('thing', 'NN'), ('2013', 'CD'), ('full-year', 'JJ'), ('result', 'NN')]


['Pfizer helps @RareDiseases launch a new resource to educate on Gaucher disease.', 'Learn more: http://t.co/5MiN0DFaFQ'

[('this', 'DT'), ('month\x89ûªs', 'JJ'), ('white', 'JJ'), ('paper', 'NN'), ('explores', 'NNS'), ('barclays', 'VBZ'), ('new', 'JJ'), ('entrepreneurs', 'NNS'), ('index', 'NN'), ('which', 'WDT'), ('map', 'VBZ'), ('business', 'NN'), ('activity', 'NN'), ('and', 'CC'), ('wealth', 'NN'), ('creation', 'NN'), ('.', '.')]


['1st panel session starting now  Creating Shared Value: beyond philantropy &amp  corporate social responsibility  http://t.co/8Z57RcCC #Nestlecsv']
[('1st', 'CD'), ('panel', 'NN'), ('session', 'NN'), ('starting', 'VBG'), ('now', 'RB'), ('creating', 'VBG'), ('shared', 'VBN'), ('value', 'NN'), (':', ':'), ('beyond', 'IN'), ('philantropy', 'NN'), ('&', 'CC'), ('amp', 'JJ'), ('corporate', 'JJ'), ('social', 'JJ'), ('responsibility', 'NN'), ('urlplaceholder', 'JJ'), ('#', '#'), ('nestlecsv', 'NN')]


['J. Baensch: We rely on a global Innovation, Technology and R&D network #Nestle #NestleIR']
[('j.', 'NN'), ('baensch', 'NN'), (':', ':'), ('we', 'PRP'), ('rely', 'VBP'), ('on', 'IN')

...

/home/mickie/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


["We're supporting a Dubai public health campaign to offer women advice on steps they can take to prevent osteoporosis: http://t.co/GqZQM9SJ"]
[('we', 'PRP'), ("'re", 'VBP'), ('supporting', 'VBG'), ('a', 'DT'), ('dubai', 'JJ'), ('public', 'JJ'), ('health', 'NN'), ('campaign', 'NN'), ('to', 'TO'), ('offer', 'VB'), ('woman', 'NN'), ('advice', 'NN'), ('on', 'IN'), ('step', 'NN'), ('they', 'PRP'), ('can', 'MD'), ('take', 'VB'), ('to', 'TO'), ('prevent', 'VB'), ('osteoporosis', 'NN'), (':', ':'), ('urlplaceholder', 'NN')]


["[Blogpost]: NutriGo, Danone's project to curb malnutrition among Chinese babies in rural areas http://t.co/AC5osuehc1  cc @danonecommunity"]
[('[', 'NN'), ('blogpost', 'NN'), (']', 'NN'), (':', ':'), ('nutrigo', 'NN'), (',', ','), ('danone', 'NN'), ("'s", 'POS'), ('project', 'NN'), ('to', 'TO'), ('curb', 'VB'), ('malnutrition', 'NN'), ('among', 'IN'), ('chinese', 'JJ'), ('baby', 'NN'), ('in', 'IN'), ('rural', 'JJ'), ('area', 'NN'), ('urlplaceholder', 'NN'), ('cc', 'NN'

[('data', 'NNS'), ('for', 'IN'), ('#', '#'), ('nsclc', 'NN'), ('to', 'TO'), ('be', 'VB'), ('presented', 'VBN'), ('on', 'IN'), ('patient-reported', 'JJ'), ('symptom', 'NN'), (',', ','), ('global', 'JJ'), ('qol', 'NN'), ('&', 'CC'), ('amp', 'JJ'), ('general', 'JJ'), ('health', 'NN'), ('status', 'NN'), ('compared', 'VBN'), ('#', '#'), ('3.400', 'CD'), ('&', 'CC'), ('amp', 'VBD'), ('3.412/hall', 'CD'), ('4', 'CD')]


['.', "@Michelle9647 We're not draining Pakistan\x89Ûªs water.", 'We firmly believe access to water is a human right http://t.co/4NkwV8oLTj']
[('.', '.')]


['Our #iPad app has been included in an interesting list of top investor relations apps http://bit.ly/dSlqvL #IR']
[('our', 'PRP$'), ('#', '#'), ('ipad', 'NN'), ('app', 'NN'), ('ha', 'NN'), ('been', 'VBN'), ('included', 'VBN'), ('in', 'IN'), ('an', 'DT'), ('interesting', 'JJ'), ('list', 'NN'), ('of', 'IN'), ('top', 'JJ'), ('investor', 'NN'), ('relation', 'NN'), ('apps', 'IN'), ('urlplaceholder', 'JJ'), ('#', '#'), ('ir', '

[('treasury', 'NN'), ('pick', 'NN'), ('tries', 'NNS'), ('to', 'TO'), ('cast', 'VB'), ('his', 'PRP$'), ('history', 'NN'), ('a', 'DT'), ('right', 'NN'), ('for', 'IN'), ('the', 'DT'), ('job', 'NN'), (':', ':'), ('jacob', 'NN'), ('lew', 'NN'), (',', ','), ('who', 'WP'), ('face', 'VBP'), ('a', 'DT'), ('senate', 'NN'), ('pa', 'NN'), ('...', ':'), ('urlplaceholder', 'JJ'), ('#', '#'), ('citigroup', 'NN'), ('#', '#'), ('brk', 'NN')]


['Thanks - you too!', 'RT @GaboNYC On our way to #Brooklyn to join #CitiVolunteers in revitalizing a school.', 'Have a great Global Community Day!', '!']
[('thanks', 'NNS'), ('-', ':'), ('you', 'PRP'), ('too', 'RB'), ('!', '.')]


["Did you know we support nearly 300 initiatives which support United Nation's Millennium Development Goals?", 'http://t.co/jxjAvSaD']
[('did', 'VBD'), ('you', 'PRP'), ('know', 'VBP'), ('we', 'PRP'), ('support', 'VBP'), ('nearly', 'RB'), ('300', 'CD'), ('initiative', 'NN'), ('which', 'WDT'), ('support', 'NN'), ('united', 'VBD'), ('natio

[('barclays', 'NNS'), ('announces', 'NNS'), ('that', 'WDT'), ('irene', 'VBP'), ('mcdermott', 'RBS'), ('brown', 'JJ'), ('ha', 'NN'), ('been', 'VBN'), ('appointed', 'VBN'), ('a', 'DT'), ('group', 'NN'), ('human', 'JJ'), ('resources', 'NNS'), ('director', 'NN'), ('urlplaceholder', 'NN')]


[" With the help of Fair Labor Association we've identified 11 areas of recommendations to tackle child labour issue.", 'http://t.co/76Ttq66p']
[('with', 'IN'), ('the', 'DT'), ('help', 'NN'), ('of', 'IN'), ('fair', 'JJ'), ('labor', 'NN'), ('association', 'NN'), ('we', 'PRP'), ("'ve", 'VBP'), ('identified', 'VBN'), ('11', 'CD'), ('area', 'NN'), ('of', 'IN'), ('recommendation', 'NN'), ('to', 'TO'), ('tackle', 'VB'), ('child', 'VB'), ('labour', 'JJ'), ('issue', 'NN'), ('.', '.')]


['Volunteers help to make a difference to rescue animals.', 'http://t.co/NupD1MAJ']
[('volunteers', 'NNS'), ('help', 'VBP'), ('to', 'TO'), ('make', 'VB'), ('a', 'DT'), ('difference', 'NN'), ('to', 'TO'), ('rescue', 'VB'), ('anim